# CHAPTER 3 딕셔너리와 집합

* 파이썬 딕셔너리 뒤에는 **해시 테이블**이라는 엔진이 있다.
* 집합도 해시 테이블을 이용해서 구현한다. 
* 해시 테이블이 작동하는 방식을 알아야 딕셔너리와 집합을 최대로 활용할 수 있따.

# 3.1 일반적인 매핑형

* `collections.abc` 모듈은 dict 및 이와 유사한 자료형의 인터페이스를 정의하기 위해 Mapping 및 MutableMapping 추상 베이스 클래스(ABC)를 제공한다. 
* 특화된 매핑은 `dict`나 `collections.UserDict`클래스를 상속하기도 한다. 
* 추상 베이스 클래스는 매핑이 제공해야 하는 최소한의 인터페이스를 정의하고 문서화하기 위한 것.
* 넓은 의미의 매핑을 지원해야 하는 코드에서 `isinstance()` 테스트를 하기 위한 기준으로 사용.

In [2]:
my_dict = {}
import collections
isinstance(my_dict, collections.abc.Mapping)

True

In [3]:
my_list = []
isinstance(my_list, collections.abc.Mapping)

False

* 표준 라이브러리에서 제공하는 매핑형은 모두 `dict`를 이용해서 구현하므로, 키가 **해시 가능**해야 한다는 제한을 갖고 있다. (값은 해시 가능할 필요 없고, 키만 해시 가능하면 된다.)

In [4]:
# 딕셔너리를 구현하는 다양한 방법
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

# 3.2 지능형 딕셔너리

* 지능형 딕셔너리(dictcomp)는 모든 반복형 객체에서 키-값 쌍을 생성함으로써 딕셔너리 객체를 만들 수 있다.

In [5]:
DIAL_CODES = [
            (86, 'China'),
            (91, 'India'),
            (1, 'United States'),
            (62, 'Indonesia'),
            (55, 'Brazil'),
            (92, 'Pakistan'),
            (880, 'Bangladesh'),
            (234, 'Nigeria'),
            (7, 'Russia'),
            (81, 'Japan')
] # dict 생성자에 키-값 쌍의 리스트를 바로 사용할 수 있따.

In [6]:
country_code = {country: code for code, country in DIAL_CODES} # 쌍을 뒤바꿔서 country는 키, code는 값

In [7]:
country_code

{'China': 86,
 'India': 91,
 'United States': 1,
 'Indonesia': 62,
 'Brazil': 55,
 'Pakistan': 92,
 'Bangladesh': 880,
 'Nigeria': 234,
 'Russia': 7,
 'Japan': 81}

In [8]:
{code: country.upper() for country, code in country_code.items() if code < 66}
# 쌍을 한 번 더 뒤바꿔서 값을 대문자로 바꾸고, code가 66보다 작은 항목만 걸러낸다.

{1: 'UNITED STATES', 62: 'INDONESIA', 55: 'BRAZIL', 7: 'RUSSIA'}

# 3.3 공통적인 매핑 메서드

* `d.update(m, [**kargs])` : `update()` 메서드가 첫 번째 인수 m을 다루는 방식은 **덕 타이핑(duck typing)** 의 대표적인 사례다. 
* 먼저 m이 `keys()` 메서드를 갖고 있는지 확인한 후, 만약 메서드를 갖고 있으면 매핑이라고 간주한다.
* `keys()` 메서드가 없으면, `update()` 메서드는 m의 항목들이 (키, 값) 쌍으로 되어있다고 간주하고 m을 반복한다. 
* 대부분의 파이썬 매핑은 `update()` 메서드와 같은 논리를 내부적으로 구현한다. 
* 따라서 매핑은 다른 매핑으로 초기화하거나, (키, 값) 쌍을 생성할 수 있는 반복형 객체로 초기화할 수 있다.

## 3.3.1 존재하지 않는 키를 setdefault()로 처리하기

* **조기 실패(fail-fast)** 철학에 따라, 존재하지 않는 키 `k`로 `d[k]`를 접근하면 `dict`는 오류를 발생시킨다. 
* `KeyError`를 처리하는 것보다 기본값을 사용하는 것이 더 편리한 경우에는 `d[k]` 대신 `d.get(k, default)`를 사용한다. 
* [예제 3-2]는 존재하지 않는 키를 처리할 때 `dict.get()`이 좋지 않은 사례를 보여준다.

In [10]:
# 예제 3-2 dict.get()을 이용해서 인덱스에서 발생한 단어 목록을 가져와서 갱신하는 index0.py.

In [16]:
"""단어가 나타나는 위치를 가리키는 인덱스를 만든다."""

import sys
import re

WORD_RE = re.compile(r'\w+')

index = {}

with open(sys.argv[0], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
#         print(line_no, line)
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            # 보기 좋은 코드는 아니지만, 설명하기 위해 이렇게 구현했다.
            
            # 단어(word)에 대한 occurrences 리스트를 가져오거나, 단어가 없으면 빈 배열([])을 가져온다. 
            occurrences = index.get(word, []) 
            occurrences.append(location)
            
            # 변경된 occurrences를 index 딕셔너리에 넣는다. 그러면 index를 한 번 더 검색한다. 
            index[word] = occurrences
            
# 알파벳순으로 출력한다.
# sorted 함수의 key 인수안에서 str.upper()를 호출하지 않고, 단지 str.upper() 함수에 대한 참조를 전달해서
# sorted 함수가 이 함수를 이용해서 정렬할 단어를 저유고하하게 만든다. 
for word in sorted(index, key=str.upper): 
    print(word, index[word])

0 [(12, 17), (13, 22)]
added [(11, 15)]
after [(4, 1)]
an [(1, 30)]
app [(15, 40), (16, 5)]
as [(15, 37)]
avoid [(3, 55)]
back [(11, 21)]
by [(11, 26)]
can [(3, 51)]
cwd [(4, 20)]
CWD [(10, 18)]
del [(13, 9)]
doing [(3, 61)]
Entry [(1, 4)]
for [(1, 16)]
from [(3, 18), (4, 24), (10, 22), (15, 5)]
if [(9, 1), (12, 5)]
import [(7, 1), (15, 20)]
imports [(3, 67)]
init_path [(11, 49)]
InteractiveShellApp [(11, 29)]
ipykernel [(3, 27), (15, 10)]
IPython [(1, 33)]
is [(3, 6), (11, 12)]
kernel [(1, 41)]
kernelapp [(15, 27)]
launching [(1, 20)]
launch_new_instance [(16, 9)]
load [(10, 45)]
package [(3, 37)]
path [(4, 33), (10, 31), (12, 12), (13, 17)]
point [(1, 10)]
Remove [(10, 7)]
removing [(4, 7)]
separate [(3, 9)]
so [(3, 45)]
stuff [(10, 50)]
sys [(4, 29), (7, 8), (10, 27), (12, 8), (13, 13)]
the [(3, 23), (4, 16), (10, 14)]
This [(3, 1), (11, 7)]
until [(3, 75)]
we [(3, 48), (10, 42)]
while [(10, 36)]
__main__ [(9, 17)]
__name__ [(9, 4)]


In [20]:
# 예제 3-4 인덱스에서 발생한 단어 목록을 가져와서 갱신하는 index.py, dict.setdefault()를 사용해서
# 단 한줄로 구현했다. 

"""단어가 나타나는 위치를 가리키는 인덱스를 만든다."""

import sys
import re

WORD_RE = re.compile(r'\w+')

index = {}

with open(sys.argv[0], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() +1
            location = (line_no, column_no)
            
            # setdefault()가 값을 반환하므로 한 번 더 검색할 필요 없이 갱신할 수 있다. 
            index.setdefault(word, []).append(location) 
            
# 알파벳순으로 출력한다,
for word in sorted(index, key=str.upper):
    print(word, index[word])

0 [(12, 17), (13, 22)]
added [(11, 15)]
after [(4, 1)]
an [(1, 30)]
app [(15, 40), (16, 5)]
as [(15, 37)]
avoid [(3, 55)]
back [(11, 21)]
by [(11, 26)]
can [(3, 51)]
cwd [(4, 20)]
CWD [(10, 18)]
del [(13, 9)]
doing [(3, 61)]
Entry [(1, 4)]
for [(1, 16)]
from [(3, 18), (4, 24), (10, 22), (15, 5)]
if [(9, 1), (12, 5)]
import [(7, 1), (15, 20)]
imports [(3, 67)]
init_path [(11, 49)]
InteractiveShellApp [(11, 29)]
ipykernel [(3, 27), (15, 10)]
IPython [(1, 33)]
is [(3, 6), (11, 12)]
kernel [(1, 41)]
kernelapp [(15, 27)]
launching [(1, 20)]
launch_new_instance [(16, 9)]
load [(10, 45)]
package [(3, 37)]
path [(4, 33), (10, 31), (12, 12), (13, 17)]
point [(1, 10)]
Remove [(10, 7)]
removing [(4, 7)]
separate [(3, 9)]
so [(3, 45)]
stuff [(10, 50)]
sys [(4, 29), (7, 8), (10, 27), (12, 8), (13, 13)]
the [(3, 23), (4, 16), (10, 14)]
This [(3, 1), (11, 7)]
until [(3, 75)]
we [(3, 48), (10, 42)]
while [(10, 36)]
__main__ [(9, 17)]
__name__ [(9, 4)]


변경된 부분을 확인해보자.

In [ ]:
my_dict.setdefault(key, []).append(new_value)

위 코드는 아래 코드를 실행한 결과와 같다.

In [ ]:
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)

# 3.4 융통성 있게 키를 조회하는 매핑

* 검색할 때 키가 존재하지 않으면 어떤 특별한 값을 반환하는 매핑이 있으면 편리한 때가 종종있다.


* 이런 딕셔너리를 만드는 방법은 크게 두 가지다.  
1) 평범한 `dict` 대신 `defaultdict`을 사용하는 방법.  
2) `dict` 등의 매핑형을 상속해서 `__mising__()` 메서드를 추가하는 방법.

## 3.4.1 defaultdict: 존재하지 않는 키에 대한 또 다른 처리

* `defaultdict`는 존재하지 않는 키로 검색할 때 요청에 따라 항목을 생성하도록 설정되어 있다.
* 작동방식은, `defaultdict` 객체를 생성할 때 존재하지 않는 키 인수로 `__getitem__()` 메서드를 호출할 때마다  
    기본값을 생성하기 위해 사용되는 콜러블을 제공하는 것이다.
      
      
* 예를 들어, `dd = defaultdict(list)` 객체를 생성한 후,   
    `dd`에 존재하지 않는 키인 `new-key`로 `dd[new-key]` 표현식을 실행하면 다음과 같이 처리된다.  
    1) 리스트를 새로 생성하기 위해 `list()`를 호출한다.  
    2) `new-key`를 키로 사용해서 새로운 리스트를 `dd`에 삽입한다.  
    3) 리스트에 대한 참조를 반환한다.
* 기본값을 생성하는 콜러블은 `default_factory`라는 객체 속성에 저장된다.

In [22]:
# 예제 3-5 index_default.py: setdefault() 메서드 대신 defaultdict 객체 사용하기
"""단어가 나타나는 위치를 가리키는 인덱스를 만든다."""
import sys
import re
import collections

WORD_RE = re.compile(r'\w+')

index = collections.defaultdict(list) # default_factory에 list생성자를 갖고 있는 defaultdict를 생성한다. 
with open(sys.argv[0], encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            
            # word가 index에 들어있지 않으면 default_factory를 호출해서 없는 값에 대한 항목을 생성하는데,
            # 여기서는 빈 리스트를 생성해서 index[word]에 할당한 후 반환하므로, 
            # append(location) 연산은 언제나 성공한다. 
            index[word].append(location)
            
# 알파벳순으로 출력한다.
for word in sorted(index, key=str.upper):
    print(word, index[word])

0 [(12, 17), (13, 22)]
added [(11, 15)]
after [(4, 1)]
an [(1, 30)]
app [(15, 40), (16, 5)]
as [(15, 37)]
avoid [(3, 55)]
back [(11, 21)]
by [(11, 26)]
can [(3, 51)]
cwd [(4, 20)]
CWD [(10, 18)]
del [(13, 9)]
doing [(3, 61)]
Entry [(1, 4)]
for [(1, 16)]
from [(3, 18), (4, 24), (10, 22), (15, 5)]
if [(9, 1), (12, 5)]
import [(7, 1), (15, 20)]
imports [(3, 67)]
init_path [(11, 49)]
InteractiveShellApp [(11, 29)]
ipykernel [(3, 27), (15, 10)]
IPython [(1, 33)]
is [(3, 6), (11, 12)]
kernel [(1, 41)]
kernelapp [(15, 27)]
launching [(1, 20)]
launch_new_instance [(16, 9)]
load [(10, 45)]
package [(3, 37)]
path [(4, 33), (10, 31), (12, 12), (13, 17)]
point [(1, 10)]
Remove [(10, 7)]
removing [(4, 7)]
separate [(3, 9)]
so [(3, 45)]
stuff [(10, 50)]
sys [(4, 29), (7, 8), (10, 27), (12, 8), (13, 13)]
the [(3, 23), (4, 16), (10, 14)]
This [(3, 1), (11, 7)]
until [(3, 75)]
we [(3, 48), (10, 42)]
while [(10, 36)]
__main__ [(9, 17)]
__name__ [(9, 4)]


* default_factory가 설정되어 있지 않으면, 키가 없을 때 흔히 볼 수 있는 `KeyError`가 발생한다.

### CAUTION_
* `defaultdict`의 `default_factory`는 `__getitem__()` 호출에 대한 기본값을 제공하기 위해 호출되며,  
    다른 메서드를 통해서는 호출되지 않는다.
* 예를 들어, `dd`가 `defaultdict` 형이며, `k`가 존재하지 않는 키면, `dd[k]`는 `default_factory`를 호출해서  
    기본값을 생성하지만, `dd.get(k)`는 단지 `None`을 반환할 뿐이다.

## 3.4.2 __missing__() 메서드

* 매핑형은 이름으로도 쉽게 추측할 수 있는 `__missing__()` 메서드를 이용해서 존재하지 않는 키를 처리한다.
* 이 특수 메서드는 기본 클래스인 `dict`에는 정의되어 있지 않지만, `dict`는 이 메서드를 알고 있다.
* 따라서, `dict` 클래스를 상속하고 `__missing__()` 메서드를 정의하면,  
    `dict.__getitem__()` 표준 메서드가 키를 발견할 수 없을 때 `KeyError`를 발생시키지 않고 `__missing__()` 메서드를 호출한다.

In [56]:
# 예제 3-6 비문자열 키를 검색할 때 키를 발견하지 못하면 키를 문자열로 반환하는 StrKeyDict0
d = StrKeyDict0([('2', 'two'), ('4', 'four')])

# d[key] 표기법을 이용해서 항목을 가져오는 테스트
d['2']

'two'

In [57]:
d[4]

'four'

In [58]:
d[1]

KeyError: '1'

In [59]:
# d.get(key) 표기법을 이용해서 항목을 가져오는 테스트
d.get('2')

'two'

In [60]:
d.get(4)

'four'

In [61]:
d.get(1, 'N/A') # 먼저 get에서 __getitem__() 실행 -> 실패 -> __missing__() 실행 -> 실패 -> return default

'N/A'

In [62]:
# in 연산자 테스트
2 in d

True

In [63]:
1 in d

False

In [55]:
# 예제 3-7 조회할 때 키를 문자열로 반환하는 StrKeyDict0
class StrKeyDict0(dict): # dict.__getitem__() 내장 메서드가 __missing__() 메서드를 지원한다는 것을 보여주기 위해
                         # dict 클래스를 상속했다.
    def __missing__(self, key):
        if isinstance(key, str): # 키가 문자열인지 확인한다. 
            raise KeyError(key)
        return self[str(key)]    # 키에서 문자열을 만들고 조회한다. 
    
    def get(self, key, default=None):
        try:
            return self[key]     # get() 메서드는 self[key] 표기법을 이용해서 __getitem__() 메서드에 위임한다.
        except KeyError:         # 이렇게 함으로써 __missing__() 메서드가작동할 수 있는 기회를 준다. 
            return default       # KeyError가 발생하면 __missing__() 메서드가 실패한 것이므로 default를 반환.
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys() 
        # 수정하지 않은 (문자열이 아닐 수 있는) 키를 검색하고 나서, 키에서 만든 문자열로 검색한다. 

### Note
* 사용자 정의 매피형을 만들 때는 `dict`보다 `collections.UserDict` 클래스를 상속하는 것이 더 낫다.

* 위에서 `__missing__()` 메서드 안에 `isinstance(key, str)` 이 없으면 `str(k)` 키가 존재하지 않으면 무한히 재귀적으로 호출된다. 
* 마지막 줄의 `self[str(key)]`가 `str` 키를 이용해서 `__getitem__()` 메서드를 호출하고,  
    이때 키가 없으면 `__missing__()` 메서드를 다시 호출하기 때문이다. 


* 이 예제가 일관성 있게 동작하려면 `__contains__()` 메서드도 필요하다. 
* `k in d` 연산이 `__contains__()` 메서드를 호출하지만,  
    `dict`에서 상속받은 `__contains__()` 메서드가 `__missing__()`을 호출하지 않기 떄문이다. 
* 키를 검색할 때 일반적으로 파이썬스러운 스타일인 `k in my_dict`로 조회하지 않는다.
* `str(key) in self` 표현식을 사용하면 재귀적으로 `__contains__()` 메서드를 호출하기 때문이다.
* 재귀적 호출 문제를 피하기 위해 여기서는 `key in self.keys()`와 같이 명시적으로 키를 조회한다.

**NOTE**  
* 파이썬3에서는 아주 큰 매핑의 경우에도 `k in my_dict.keys()` 형태의 검색이 효율적이다.

# 3.5 그 외 매핑형

**collections.OrderedDict**
* 키를 삽입한 순서대로 유지함으로써 항목을 반복하는 순서를 예측할 수 있다.

**collections.ChainMap**
* 매핑들의 목록을 담고 있으며 한꺼번에 모두 검색할 수 있다. 각 매핑을 차례대로 검색하고,  
    그중 하나에서라도 키가 검색되면 성공한다. 
    
**collcetions.Counter**
* 모든 키에 정수형 카운터를 갖고 있는 매핑.
* 기존 키를 갱신하면 카운터가 늘어난다. 

# 3.6 UserDict 상속하기

* `dict`보다는 `UserDict`를 상속해서 매핑형을 만드는 것이 쉽다.
* `UserDict`는 `dict`을 상속하지 않고 내부에 실제 항목을 담고 있는 `data`라고 하는 `dict`객체를 갖고 있다.
* 이렇게 구현함으로써 `__setitem__()` 등의 특수 메서드를 구현할 때 발생하는 원치않는 재귀적 호출을 피할 수 있으며,
* `__count__()` 메서드를 간단히 구현할 수 있다.

In [64]:
# StrKeyDict는 모든 키를 str 형으로 저장함으로써 비문자열 키로 객체를 생성하거나
# 갱신할 때, 발생할 수 있는 예기치 못한 문제를 피하게 해준다.
# 예제 3-8 삽입, 갱신, 조회할 때 비문자열 키를 항상 문자열로 변환하는 StrKeyDict

import collections

class StrKeyDict(collections.UserDict):  # StrKeyDict는 UserDict를 상속한다. 
    
    def __missing__(self, key):          # __missing__() 메서드는 [예제 3-7]과 완전히 똑같다.
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data     # __contains__() 메서드는 더 간단하다. 저장된 키가 모두 str형이므로,
                                         # StrKeyDict0에서 self.keys()를 호출하는 방법과 달리 
                                         # self.data에서 바로 조회할 수 있다. 
    def __setitem__(self, key, item):
        self.data[str(key)] = item       # __setitem__() 메서드는 모든 키를 str형으로 변환하므로, 
                                         # 연산을 self.data에 위임할 때 더 간단히 작성할 수 있다. 

* `Userdict` 클래스가 `MutableMapping`을 상속하므로 `StrKeyDict`는 결국 `UserDict`,   
    `MutableMapping` 또는 `Mapping`을 상속하게 되어 매핑의 모든 기능을 가지게 된다. 
* `Mapping`은 추상 베이스 클래스(ABC)임에도 불구하고 유용한 구상(concrete) 메서드를 다수 제공한다. 

**MutableMapping.update()**
* 이 강력한 메서드는 직접 호출할 수도 있지만,   
    다른 매핑이나 (키, 값)쌍의 반복형 및 키워드 인수에서 객체를 로딩하기 위해 `__init__()`에 의해 사용될 수도 있다. 
* 이 메서드는 항목을 추가하기 위해 'self[키] = 값' 구문을 사용하므로 결국 서브클래스에서 구현한 `__setitem__()` 메서드를 호출하게 된다.

**Mapping.get()**
* StrKeyDict0(예제 3-7)에서는 `__getitem__()`과 일치하는 결과를 가져오기 위해 `get()` 메서드를 직접 구현해야 했지만, 
* [예제 3-8]에서는 `StryKeyDict0.get()`과 완전히 동일하게 구현된 `Mapping.get()`을 상속받는다.

# 3.7 불변 매핑

* 사용자가 실수로 매핑을 변경하지 못하도록 보장하고 싶은 경우.
* 파이썬3.3 이후 `types` 모듈은 `MappingProxyType`이라는 래퍼 클래스를 제공해서,   
    원래 매핑의 동적인 뷰를 제공하지만 읽기 전용의 `mappingproxy` 객체를 반환한다.

In [65]:
# 예제 3-9 dict에서 읽기 전용 mappingproxy 객체를 생성하는 MappingProxyType
from types import MappingProxyType

d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [66]:
d_proxy[1]

'A'

In [67]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [68]:
d[2] = 'B'

In [69]:
d_proxy

mappingproxy({1: 'A', 2: 'B'})

In [70]:
d_proxy[2]

'B'

# 3.8 집합 이론

In [71]:
l = ['spam', 'spam', 'eggs', 'spam']

In [72]:
set(l)

{'eggs', 'spam'}

In [73]:
list(set(l))

['spam', 'eggs']

* 고유함을 보장하는 것 외에 집합형은 중위 연산자를 이용해서 기본적인 집합 연산을 구현한다.
* 두 개의 집합 a, b에 대해서 `a | b`는 합집합, `a & b`는 교집합, `a - b`는 차집합을 계산한다.

In [82]:
# 예제 3-10 둘 다 집합형인 haystack 안에 들어 있는 needles 항목 수 구하기
haystack = set(['a@gmail.com', 'b@gmail.com', 'c@gmail.com'])
needles = set(['a@gmail.com', 'c@gmail.com'])

found = len(needles & haystack)
found

2

In [83]:
# 예제 3-11 haystack 안에서 needles의 발생 횟수 구하기
found = 0
for n in needles:
    if n in haystack:
        found += 1

In [84]:
found

2

* [예제 3-10]의 코드는 두 객체가 모두 집합이어야 하는 반면, [예제 3-11]은 iterable한 객체라면 사용할 수 있다.
* 그러나 객체가 집합형이 아니어도 [예제 3-12]와 같이 즉석에서 집합으로 만들 수 있다.

In [4]:
# 예제 3-12 haystack 안에서 needles의 발생 횟수 구하기
haystack = ['a@gmail.com', 'b@gmail.com', 'c@gmail.com']
needles = ['a@gmail.com', 'c@gmail.com']

found = len(set(needles) & set(haystack))
print(found)

# 또 다른 방법:
found = len(set(needles).intersection(haystack))
print(found)

2
2


## 3.8.1 집합 리터럴
* `{1}`, `{1, 2}` 등 집합 리터럴에 대한 구문은 수학적 표기법과 동일하지만,  
    공집합은 리터럴로 표기할 수 없으므로, 반드시 `set()`으로 표기해야 한다.

* 파이썬3 에서는 공집합 이외의 집합을 표준 문자열로 표현하기 위해 언제나 `{}`구문을 사용한다.

In [5]:
s = {1}
type(s)

set

In [6]:
s

{1}

In [7]:
s.pop()

1

In [8]:
s

set()

* `{1, 2, 3}`과 같은 리터럴 집합 구문은 `set([1, 2, 3])` 처럼 생성자를 호출하는 것보다 더 빠르고 가독성이 좋다.

In [10]:
# 디스어셈블러 함수인 dis.dis()를 이용해서 두 개의 연산에 대한 바이트코드를 살펴보자.
from dis import dis
dis('{1}') # 리터럴 집합 구문은 BUILD_SET이라는 특수 바이트코드를 실행한다. 

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [11]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


* 리터럴 집합 구문에서 `BUILD_SET` 특수 바이트코드가 거의 모든 일을 처리한다.


* `frozenset`에 대한 별도의 리터럴 구문은 없으며, `frozenset`은 언제나 생성자를 호출해서 생성해야 한다.

In [13]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

## 3.8.2 지능형 집합

In [17]:
# 예제 3-13 유니코드명 안에 'SIGN'이 들어 있는 단어를 가진 Latin-1 문자들의 집합 만들기
from unicodedata import name
print({chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')})
# 지능형 딕셔너리와 뭐가 다른 것일까...

{'±', '+', '=', '%', '°', '®', '<', '$', '#', '¬', '¶', '©', '§', '¥', 'µ', '>', '÷', '×', '¤', '£', '¢'}


## 3.8.3 집합 연산
**책 참고**

# 3.9 dict와 set의 내부 구조

* 파이썬 dict와 set은 얼마나 효율적인가?
* 왜 순서가 없을까?
* dict의 키와 set 항목에 파이썬의 모든 객체를 사용할 수 없는 이유는 무엇인가?
* dict의 키와 set 항목의 순서가 왜 삽입 순서에 따라 달라지며, 객체 수명주기 동안 이 순서가 바뀔 수 있는 이유는 무엇일까?
* 딕셔너리와 집합을 반복하는 동안 항목을 추가하면 왜 안 될까?

## 3.9.1 성능 실험

**결과 요약** : dict이 제일 빠름 < set 큰 차이 없음 < list 시간 오래 걸림

## 3.9.2 딕셔너리 안의 해시 테이블

### 해시와 동치성
* `hash()` 내장 함수는 내장 자료형은 직접 처리하고 사용자 정의 자료형의 경우 `__hash__()` 메서드를 호출한다. 
* 두 객체가 동일하면 이 값들의 해시값도 동일해야 한다. 

### 해시 테이블 알고리즘
1) `my_dict[search_key]`에서 값을 가져오기 위해 파이썬은 `__hash__(search_key)`를 호출해 `search_key`의 해시값을 가져온다.   
2) 해시값의 최하위 비트를 해시 테이블 안의 버킷에 대한 오프셋으로 사용한다.  
3) 찾아낸 버킷이 비어 있으면 `KeyError`를 발생시키고,  
4) 그렇지 않으면 버킷에 들어 있는 항목인 `(found_key: found_value)`쌍을 검사해서 `search_key == found_key`인지 검사한다.  
5) 이 값이 일치하면 항목을 찾은 것이므로 `found_value`를 반환한다.

![플로차트](./ch3_1.png)

## 3.9.3 dict 작동 방식에 의한 영향

### 키 객체는 반드시 해시 가능해야 한다.
1) 객체의 수명 주기 동안 언제나 동일한 값을 반환하는 `__hash__()` 메서드를 제공해서 `hash()`함수를 지원한다.  
2) `__eq__()` 메서드를 통해 동치성을 판단할 수 있다.  
3) `a == b`가 참이면, `hash(a) == hash(b)` 도 반드시 참이어야 한다.

### dict의 메모리 오버헤드가 크다.
* dict가 내부적으로 해시 테이블을 사용하고 있고 해시가 제대로 작동하려면 빈 공간이 충분해야 하므로,  
    dict의 메모리 공간 효율성은 높지 않다. 

### 키 검색이 아주 빠르다.
* dict는 속도를 위해 공간을 포기하는 예다. 

### 키 순서는 삽입 순서에 따라 달라진다.
* 해시 충돌이 발생하면 두 번째 키는 충돌이 발생하지 않았을 때의 정상적인 위치와 다른 곳에 놓이게 된다. 

### 딕셔너리에 항목을 추가하면 기존 키의 순서가 변경될 수 있다
* dict에 항목을 추가할 때마다 파이썬 인터프리터는 그 딕셔너리의 해시 테이블의 크기를 늘릴지 판단한다.
* 따라서 이 과정 동안 기존과 다르게 해시 충돌이 발생해 새로운 해시 테이블에서의 키 순서가 달라질 수 있다.
* 딕셔너리를 검색하면서 항목을 추가해야 하는 경우에는 다음 두 단계로 수행한다.  
    1) 처음부터 끝까지 딕셔너리를 검색하면서 필요한 항목은 별도의 딕셔너리에 추가한다.  
    2) 별도의 딕셔너리로 원래 딕셔너리를 갱신한다.

## 3.9.4 집합의 작동 방식 - 현실적으로 미치는 영향

* set과 frozenset도 해시 테이블을 이용해서 구현하지만, 각 버킷이 항목에 대한 참조만을 담고 있다는 점이 다르다.